In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
from dbfread import DBF
import jenkspy
import json
from jenkspy import jenks_breaks

In [2]:
#RUTAS ENTRADA
MZA_2020_ruta = r"C:\Users\ADMIN\Desktop\Ejercicio\Base de Datos Censal\MZNA_2020_NAC.csv"
NSE_ruta = r"C:\Users\ADMIN\Desktop\Ejercicio\NSE\nse_ageb_amai.xlsx"
CVE_METRO_ruta = r"C:\Users\ADMIN\Desktop\Ejercicio\ClavesMetro.csv"

# ruta shp base
SHP_MZA_NAC_ruta = r"C:\Users\ADMIN\Desktop\Ejercicio\MZN_NAC"

#ruta gpkg fin
MAPA_TIPO_ruta = r"C:\Users\ADMIN\Desktop\Ejercicio\NSE\NSE_MZA_NAC.gpkg"

In [6]:
# Cargar archivo en un DataFrame
MZNA_2020_df = pd.read_csv(MZA_2020_ruta, dtype=str)

# Leer shp base
MAPA_TIPO_gdf = gpd.read_file(SHP_MZA_NAC_ruta)

# Filtrar censos
MZNA_2020_df_F = MZNA_2020_df[['CVEGEO', 'POBTOT','PEA']]

# Unir campos de los DataFrames al GeoPackage base por CVEGEO
MAPA_TIPO_gdf1 = MAPA_TIPO_gdf.merge(MZNA_2020_df_F[['CVEGEO', 'POBTOT','PEA']], on='CVEGEO', how='left')

# reemplazar * con 1
def replace(dataframe):
    return dataframe.replace('*', '1')

# limpiar  *
MAPA_TIPO_gdf4_A = replace(MAPA_TIPO_gdf1)

# reemplazar N/D
MAPA_TIPO_gdf4_B = MAPA_TIPO_gdf4_A.replace('N/D', np.nan)

# Actualizar valores sin información de "null" a 0
MAPA_TIPO_gdf4_C = MAPA_TIPO_gdf4_B.fillna(0)

In [7]:
#Subir archivo con CVE_METRO
CVE_METRO_df = pd.read_csv(CVE_METRO_ruta, dtype={'CVE_MET': str, 'CVEMUN':str})

MAPA_TIPO_gdf = MAPA_TIPO_gdf4_C

#Crear campo CVEMUN
def homologate_municipio_id(row):
    return f"{row['CVE_ENT']}{row['CVE_MUN']}"

MAPA_TIPO_gdf['CVEMUN'] = MAPA_TIPO_gdf.apply(homologate_municipio_id, axis=1)

#Añadir claves a GPKG
MAPA_TIPO_gdf = MAPA_TIPO_gdf.merge(CVE_METRO_df[['CVEMUN', 'CVE_MET', 'NOM_MET', 'TIPO_MET']], on='CVEMUN', how='left')

#FILTRAR MUNICIPIOS ZONAS METROPOLITANAS
valores_filtro = CVE_METRO_df['CVEMUN'].unique()

MAPA_TIPO_gdf = MAPA_TIPO_gdf[(MAPA_TIPO_gdf['CVEMUN'].isin(valores_filtro))]

# Guardar el GeoPackage base actualizado
MAPA_TIPO_gdf.to_file(MAPA_TIPO_ruta, layer= 'BASE_DATOS_METRO', driver="GPKG")

In [ ]:
# Leer el archivo Excel
nse_df = pd.read_excel(NSE_ruta, usecols=['CVEAGEB', 'PONDERADO', 'PREDOMINANTE'])

# Leer la capa específica del GeoPackage
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='BASE_DATOS_METRO')

In [10]:
# Crear el campo CVEAGEB en el GeoDataFrame concatenando los campos necesarios
MAPA_TIPO_gdf['CVEAGEB'] = MAPA_TIPO_gdf['CVE_ENT'].astype(str) + MAPA_TIPO_gdf['CVE_MUN'].astype(str) + MAPA_TIPO_gdf['CVE_LOC'].astype(str) + MAPA_TIPO_gdf['CVE_AGEB'].astype(str)

# Realizar la unión (merge) por el campo CVEAGEB
merged_gdf = MAPA_TIPO_gdf.merge(nse_df, on='CVEAGEB')

# Guardar el resultado en un nuevo archivo (por ejemplo, en un GeoPackage)
merged_gdf.to_file(MAPA_TIPO_ruta, layer= 'NSE_NAC_MZA', driver="GPKG")

print("Proceso completado y el archivo guardado.")

Proceso completado y el archivo guardado.


In [11]:
# Leer la capa específica del GeoPackage
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='NSE_NAC_MZA')

In [ ]:
# Convertir las columnas 'PEA', 'POBTOT' y 'PONDERADO' a numéricas
MAPA_TIPO_gdf[['PEA', 'POBTOT', 'PONDERADO']] = MAPA_TIPO_gdf[['PEA', 'POBTOT', 'PONDERADO']].apply(pd.to_numeric, errors='coerce')

In [18]:
# Crear nuevos campos
MAPA_TIPO_gdf['DEP_ECO'] = MAPA_TIPO_gdf['PEA'] / MAPA_TIPO_gdf['POBTOT']
MAPA_TIPO_gdf['area_ha'] = MAPA_TIPO_gdf.geometry.area / 10000 
MAPA_TIPO_gdf['DENS_DEP'] = MAPA_TIPO_gdf['DEP_ECO'] / MAPA_TIPO_gdf['area_ha']
MAPA_TIPO_gdf['CAP_ECO'] = MAPA_TIPO_gdf['DENS_DEP'] * MAPA_TIPO_gdf['PONDERADO']
MAPA_TIPO_gdf['CAP_ECOv2'] = MAPA_TIPO_gdf['DEP_ECO'] * MAPA_TIPO_gdf['PONDERADO']

print("Nuevos campos creados en MAPA_TIPO_gdf.")

Nuevos campos creados en MAPA_TIPO_gdf.


In [19]:
# Guardar el GeoPackage
MAPA_TIPO_gdf.to_file(MAPA_TIPO_ruta, layer= 'CAP_ECO_NAC_MZA', driver="GPKG")